# Getting Started

## Import Libraries

In [7]:
import pandas as pd

## Import First Dataset (Volume Per Train Station)

In [8]:
augData = './datasets/Aug_2023.csv'
septData = './datasets/Sept_2023.csv'
octData = './datasets/Oct_2023.csv'


dfAug = pd.read_csv(augData)
dfSept = pd.read_csv(septData)
dfOct = pd.read_csv(octData)

print(dfAug)
print(dfSept)
print(dfOct)

     YEAR_MONTH          DAY_TYPE  TIME_PER_HOUR PT_TYPE        PT_CODE  \
0       2023-08           WEEKDAY             22   TRAIN           NS28   
1       2023-08  WEEKENDS/HOLIDAY             22   TRAIN           NS28   
2       2023-08  WEEKENDS/HOLIDAY              0   TRAIN      DT10/TE11   
3       2023-08           WEEKDAY              0   TRAIN      DT10/TE11   
4       2023-08           WEEKDAY             10   TRAIN  EW16/NE3/TE17   
...         ...               ...            ...     ...            ...   
6815    2023-08  WEEKENDS/HOLIDAY              6   TRAIN           DT23   
6816    2023-08  WEEKENDS/HOLIDAY              7   TRAIN  NS27/CE2/TE20   
6817    2023-08           WEEKDAY              7   TRAIN  NS27/CE2/TE20   
6818    2023-08  WEEKENDS/HOLIDAY             12   TRAIN            SE5   
6819    2023-08           WEEKDAY             12   TRAIN            SE5   

      TOTAL_TAP_IN_VOLUME  TOTAL_TAP_OUT_VOLUME  
0                     752                   311  

### Combine and verify all files for first dataset

In [9]:
# Combine All DataFrames
dfCombinedFirst = pd.concat([dfAug, dfSept, dfOct], ignore_index=True)
print(dfCombinedFirst)

# To Verify Combination
totalRows = dfAug.shape[0] + dfSept.shape[0] + dfOct.shape[0]
print("Total Rows should be:", totalRows)
if dfAug.shape[1] == dfSept.shape[1] == dfOct.shape[1]:
    print("Number of columns is the same for all three DataFrames:", dfAug.shape[1])
else:
    print("Column size is not the same for the DataFrames")


      YEAR_MONTH          DAY_TYPE  TIME_PER_HOUR PT_TYPE        PT_CODE  \
0        2023-08           WEEKDAY             22   TRAIN           NS28   
1        2023-08  WEEKENDS/HOLIDAY             22   TRAIN           NS28   
2        2023-08  WEEKENDS/HOLIDAY              0   TRAIN      DT10/TE11   
3        2023-08           WEEKDAY              0   TRAIN      DT10/TE11   
4        2023-08           WEEKDAY             10   TRAIN  EW16/NE3/TE17   
...          ...               ...            ...     ...            ...   
20456    2023-10  WEEKENDS/HOLIDAY              6   TRAIN           DT23   
20457    2023-10  WEEKENDS/HOLIDAY              7   TRAIN  NS27/CE2/TE20   
20458    2023-10           WEEKDAY              7   TRAIN  NS27/CE2/TE20   
20459    2023-10           WEEKDAY             12   TRAIN            SE5   
20460    2023-10  WEEKENDS/HOLIDAY             12   TRAIN            SE5   

       TOTAL_TAP_IN_VOLUME  TOTAL_TAP_OUT_VOLUME  
0                      752          

## Preparing Our First Dataset for Analysis

### Finding Blank Columns or Rows

In [17]:
# Using isna() method to find blanks
blanks = dfCombinedFirst.isna()
print(blanks)

       YEAR_MONTH  DAY_TYPE  TIME_PER_HOUR  PT_TYPE  PT_CODE  \
0           False     False          False    False    False   
1           False     False          False    False    False   
2           False     False          False    False    False   
3           False     False          False    False    False   
4           False     False          False    False    False   
...           ...       ...            ...      ...      ...   
20456       False     False          False    False    False   
20457       False     False          False    False    False   
20458       False     False          False    False    False   
20459       False     False          False    False    False   
20460       False     False          False    False    False   

       TOTAL_TAP_IN_VOLUME  TOTAL_TAP_OUT_VOLUME  
0                    False                 False  
1                    False                 False  
2                    False                 False  
3                    False 

### Finding Duplicated Columns or Rows

In [16]:
duplicates = dfCombinedFirst.duplicated().count
print(duplicates)

<bound method Series.count of 0        False
1        False
2        False
3        False
4        False
         ...  
20456    False
20457    False
20458    False
20459    False
20460    False
Length: 20461, dtype: bool>


### Removing repeated columns

In [ ]:
# Renaming a repeating column (PT_TYPE) to PT_NAME
dfCombinedFirst = dfCombinedFirst.rename(columns={'PT_TYPE': 'PT_NAME'})

### Mapping Station Codes with Station Names

In [ ]:
# Map Station Names (PT_NAME) with Station Codes (PT_CODE)
dfCombinedFirst['PT_CODE_FirstPart'] = dfCombinedFirst['PT_CODE'].str.split('/').str[0] #Splitting Stations with multiple codes
csv_df = pd.read_csv('./datasets/TrainStationCodes.csv') # Our Train Station Names file
code_name_mapping = dict(zip(csv_df['stn_code'], csv_df['mrt_station_english'])) #Mapping
dfCombinedFirst['PT_NAME'] = dfCombinedFirst['PT_CODE_FirstPart'].map(code_name_mapping) # Mapping
dfCombinedFirst = dfCombinedFirst.drop('PT_CODE_FirstPart', axis=1) # Remove column used for mapping


print(dfCombinedFirst)

### Changing Month_Year & Time_Per_Year column to proper datetime format

In [ ]:
# Convert YEAR_MONTH to the last day of the month
dfCombinedFirst['YEAR_MONTH'] = pd.to_datetime(dfCombinedFirst['YEAR_MONTH']).dt.to_period('M').dt.to_timestamp('M') + pd.offsets.MonthEnd(0)

# Combine with TIME_PER_HOUR to create a full datetime
dfCombinedFirst['DATETIME'] = dfCombinedFirst.apply(lambda row: pd.Timestamp(year=row['YEAR_MONTH'].year,
                                                   month=row['YEAR_MONTH'].month,
                                                   day=row['YEAR_MONTH'].day,
                                                   hour=row['TIME_PER_HOUR']), axis=1)

# Drop the original YEAR_MONTH column
dfCombinedFirst.drop('YEAR_MONTH', axis=1, inplace=True)

# Make DATETIME the first column by creating a new DataFrame with the desired column order
dfCombinedFirst = dfCombinedFirst[['DATETIME'] + [col for col in dfCombinedFirst.columns if col != 'DATETIME']]

# Drop the origin Time_Per_Hour columns
dfCombinedFirst.drop('TIME_PER_HOUR', axis=1, inplace=True)




### Changing (Weekdays to 0) & (Weekends/Holidays to  1)

In [ ]:
dfCombinedFirst['DAY_TYPE'] = dfCombinedFirst['DAY_TYPE'].map({'WEEKDAY': 0, 'WEEKENDS/HOLIDAY': 1})

### Creating Train_Lines column to calculate number of train lines in the station

In [ ]:
# Define a function that counts the number of train lines
def count_train_lines(pt_code):
    if pd.isna(pt_code):
        return 0
    return pt_code.count('/') + 1

# Apply the function to the PT_CODE column and create the TRAIN_LINES column
dfCombinedFirst['TRAIN_LINES'] = dfCombinedFirst['PT_CODE'].apply(count_train_lines)

# Insert TRAIN_LINES next to PT_CODE column
loc = dfCombinedFirst.columns.get_loc('PT_NAME') + 1
dfCombinedFirst.insert(loc, 'TRAIN_LINES', dfCombinedFirst.pop('TRAIN_LINES'))


### Mapping train lines to train codes

In [ ]:
train_line_mapping = {
    'EW': 0, # East-West Line
    'CG': 0, # East-West Line to Changi Airport
    'NS': 1, # North-South Line
    'NE': 2, # North-East Line
    'CC': 3, # Circle Line
    'CE': 3, # Circle Line (Bayfront, Marina Bay)
    'DT': 4, # Downtown Line
    'TE': 5, # Thomson-East Coast Line
    'BP': 6, # Bukit Panjang LRT
    'SW': 7, # Sengkang LRT West
    'SE': 7, # Sengkang LRT East
    'PW': 8, # Punggol LRT West
    'PE': 8, # Punggol LRT East
}

def map_train_codes(pt_code):
    # Initialize an empty list to store the mapped train codes
    train_codes = []
    # Split the pt_code by '/' and iterate over each part
    for code in pt_code.split('/'):
        # Iterate over each key in the mapping to find a match
        for key in train_line_mapping:
            # If the key is found at the start of the code segment, append the mapped value
            if code.startswith(key):
                train_codes.append(train_line_mapping[key])
                break  # Break the loop once the match is found
    return train_codes

# Apply the revised function to the PT_CODE column
dfCombinedFirst['TRAIN_CODES'] = dfCombinedFirst['PT_CODE'].apply(map_train_codes)

# Convert the TRAIN_CODES column to a list
train_codes_list = dfCombinedFirst['TRAIN_CODES'].tolist()

# Insert TRAIN_CODES next to TRAIN_LINES
loc = dfCombinedFirst.columns.get_loc('TRAIN_LINES') + 1
dfCombinedFirst.insert(loc, 'TRAIN_CODES', dfCombinedFirst.pop('TRAIN_CODES'))

### Check number of train stations in Our Dataset 

In [ ]:
numberOfTrainStations = dfCombinedFirst['PT_NAME'].nunique()
print(numberOfTrainStations)

### Check number of train station in our Location Dataset

In [ ]:
dfLocation = pd.read_csv("./datasets/TrainStationLocation.csv")
unique_station_name_count = dfLocation['station_name'].nunique()
print(unique_station_name_count)

### Check that we have locations data for all of our train stations

In [ ]:
# Find PT_NAMEs that are not present in the station_name and drop duplicates
missing_stations = dfCombinedFirst[~dfCombinedFirst['PT_NAME'].isin(dfLocation['station_name'])]
missing_stations_unique = missing_stations.drop_duplicates(subset=['PT_NAME'])
 
# Get the unique missing station names as a list
missing_station_names_unique = missing_stations_unique['PT_NAME'].tolist()

print(list(missing_station_names_unique))
print("This list shows what stations is not in our location dataset.")
print(len(missing_station_names_unique))
print("This number should be: 0")

### Finding out what is the hidden 1 station in Singapore..

In [ ]:
dfLocation_set = set(dfLocation['station_name'])
dfCombinedFirst_set = set(dfCombinedFirst['PT_NAME'])


unique_station = dfLocation_set - dfCombinedFirst_set


print(unique_station)


### Mapping location data to our station names

In [ ]:
latitude_mapping = dict(zip(dfLocation['station_name'], dfLocation['lat'])) #Mapping
longitude_mappping = dict(zip(dfLocation['station_name'], dfLocation['lng'])) #Mapping
dfCombinedFirst['PT_LATITUDE'] = dfCombinedFirst['PT_NAME'].map(latitude_mapping) # Creating PT_LATITUDE column
dfCombinedFirst['PT_LONGITUDE'] = dfCombinedFirst['PT_NAME'].map(longitude_mappping) # Creating PT_LONGITUDE column

# Insert PT_LATITUDE & PT_LONGITUDE next to PT_CODE column
loc = dfCombinedFirst.columns.get_loc('PT_CODE') + 1
dfCombinedFirst.insert(loc, 'PT_LATITUDE', dfCombinedFirst.pop('PT_LATITUDE'))
loc1 = dfCombinedFirst.columns.get_loc('PT_CODE') + 2
dfCombinedFirst.insert(loc1, 'PT_LONGITUDE', dfCombinedFirst.pop('PT_LONGITUDE'))

## Saving our First Processed Dataset

In [ ]:
# Saving our processed dataset as csv
output_path = './outputDatasets/trainStationData1.csv'
dfCombinedFirst.to_csv(output_path, index=False)

## Import Second Dataset (Volume for Origin-Destination Train Station)

In [ ]:
augOriginData = './datasets/AugOrigin_2023.csv'
septOriginData = './datasets/septOrigin_2023.csv'
octOriginData = './datasets/octOrigin_2023.csv'

dfAugOrigin = pd.read_csv(augOriginData)
dfSeptOrigin = pd.read_csv(septOriginData)
dfOctOrigin = pd.read_csv(octOriginData)

print(dfAugOrigin)
print(dfSeptOrigin)
print(dfOctOrigin)

### Combine and verify all files for second dataset

In [ ]:
# Combine All DataFrames
dfCombinedSecond = pd.concat([dfAugOrigin, dfSeptOrigin, dfOctOrigin], ignore_index=True)
print(dfCombinedSecond)

# To Verify Combination
totalRows = dfAugOrigin.shape[0] + dfSeptOrigin.shape[0] + dfOctOrigin.shape[0]
print("Total Rows should be:", totalRows)
if dfAugOrigin.shape[1] == dfSeptOrigin.shape[1] == dfOctOrigin.shape[1]:
    print("Number of columns is the same for all three DataFrames:", dfAugOrigin.shape[1])
else:
    print("Column size is not the same for the DataFrames")

## Preparing Our Second Dataset for Analysis

### Removing repeated columns

In [ ]:
# Renaming a repeating column (PT_TYPE) to PT_NAME
dfCombinedSecond = dfCombinedSecond.rename(columns={'PT_TYPE': 'ORIGIN_PT_NAME'})

### Mapping Station Codes with Station Names

In [ ]:
# Map Station Names (PT_NAME) with Station Codes (PT_CODE)
#dfCombinedFirst['PT_CODE_FirstPart'] = dfCombinedFirst['PT_CODE'].str.split('/').str[0] #Splitting Stations with multiple codes
dfCombinedSecond['ORIGIN_PT_CODE_FirstPart'] = dfCombinedSecond['ORIGIN_PT_CODE'].str.split('/').str[0]
code_name_mapping = dict(zip(csv_df['stn_code'], csv_df['mrt_station_english'])) #Mapping
dfCombinedSecond['ORIGIN_PT_NAME'] = dfCombinedSecond['ORIGIN_PT_CODE_FirstPart'].map(code_name_mapping) # Mapping
dfCombinedSecond = dfCombinedSecond.drop('ORIGIN_PT_CODE_FirstPart', axis=1) # Remove column used for mapping

dfCombinedSecond['DESTINATION_PT_CODE_FirstPart'] = dfCombinedSecond['DESTINATION_PT_CODE'].str.split('/').str[0]
code_name_mapping = dict(zip(csv_df['stn_code'], csv_df['mrt_station_english'])) #Mapping
dfCombinedSecond['DESTINATION_PT_NAME'] = dfCombinedSecond['DESTINATION_PT_CODE_FirstPart'].map(code_name_mapping) # Mapping
dfCombinedSecond = dfCombinedSecond.drop('DESTINATION_PT_CODE_FirstPart', axis=1) # Remove column used for mapping

# Insert DESTINATION_PT_NAME next to ORIGIN_PT_CODE column
loc = dfCombinedSecond.columns.get_loc('ORIGIN_PT_CODE') + 1
dfCombinedSecond.insert(loc, 'DESTINATION_PT_NAME', dfCombinedSecond.pop('DESTINATION_PT_NAME'))

print(dfCombinedSecond)

### Changing (Weekdays to 0) & (Weekends/Holidays to  1)

In [ ]:
dfCombinedSecond['DAY_TYPE'] = dfCombinedSecond['DAY_TYPE'].map({'WEEKDAY': 0, 'WEEKENDS/HOLIDAY': 1})

### Creating Train_Lines column to calculate number of train lines in the station

In [ ]:
# Apply the function from count_train_lines to create ORIGIN_TRAIN_LINES column
dfCombinedSecond['ORIGIN_TRAIN_LINES'] = dfCombinedSecond['ORIGIN_PT_CODE'].apply(count_train_lines)

# Insert ORIGN_TRAIN_LINES next to ORIGIN_PT_NAME column
loc = dfCombinedSecond.columns.get_loc('ORIGIN_PT_NAME') + 1
dfCombinedSecond.insert(loc, 'ORIGIN_TRAIN_LINES', dfCombinedSecond.pop('ORIGIN_TRAIN_LINES'))

# Apply the function from count_train_lines to create DESTINATION_TRAIN_LINES column
dfCombinedSecond['DESTINATION_TRAIN_LINES'] = dfCombinedSecond['DESTINATION_PT_CODE'].apply(count_train_lines)

# Insert DESTINATION_TRAIN_LINES next to DESTINATION_PT_NAME column
loc1 = dfCombinedSecond.columns.get_loc('DESTINATION_PT_NAME') + 1
dfCombinedSecond.insert(loc1, 'DESTINATION_TRAIN_LINES', dfCombinedSecond.pop('DESTINATION_TRAIN_LINES'))

### Changing Month_Year & Time_Per_Year column to proper datetime format

In [ ]:
# Convert YEAR_MONTH to the last day of the month
dfCombinedSecond['YEAR_MONTH'] = pd.to_datetime(dfCombinedSecond['YEAR_MONTH']).dt.to_period('M').dt.to_timestamp('M') + pd.offsets.MonthEnd(0)

# Combine with TIME_PER_HOUR to create a full datetime
dfCombinedSecond['DATETIME'] = dfCombinedSecond.apply(lambda row: pd.Timestamp(year=row['YEAR_MONTH'].year,
                                                   month=row['YEAR_MONTH'].month,
                                                   day=row['YEAR_MONTH'].day,
                                                   hour=row['TIME_PER_HOUR']), axis=1)

# Drop the original YEAR_MONTH column
dfCombinedSecond.drop('YEAR_MONTH', axis=1, inplace=True)

# Make DATETIME the first column by creating a new DataFrame with the desired column order
dfCombinedSecond = dfCombinedSecond[['DATETIME'] + [col for col in dfCombinedSecond.columns if col != 'DATETIME']]

# Drop the origin Time_Per_Hour columns
dfCombinedSecond.drop('TIME_PER_HOUR', axis=1, inplace=True)



### Mapping location data to our station names

In [ ]:
latitude_mapping = dict(zip(dfLocation['station_name'], dfLocation['lat'])) #Mapping
longitude_mappping = dict(zip(dfLocation['station_name'], dfLocation['lng'])) #Mapping

# Creating ORIGIN_PT_LATITUDE & ORIGIN_PT_LONGITUDE column
dfCombinedSecond['ORIGIN_PT_LATITUDE'] = dfCombinedSecond['ORIGIN_PT_NAME'].map(latitude_mapping) 
dfCombinedSecond['ORIGIN_PT_LONGITUDE'] = dfCombinedSecond['ORIGIN_PT_NAME'].map(longitude_mappping) 

# Insert ORIGIN LATITUDE & LATITUDE next to their respective columns
locOriginLatitude = dfCombinedSecond.columns.get_loc('ORIGIN_PT_CODE') + 1
locOriginLongitude = dfCombinedSecond.columns.get_loc('ORIGIN_PT_CODE') + 2
dfCombinedSecond.insert(locOriginLatitude, 'ORIGIN_PT_LATITUDE', dfCombinedSecond.pop('ORIGIN_PT_LATITUDE'))
dfCombinedSecond.insert(locOriginLongitude, 'ORIGIN_PT_LONGITUDE', dfCombinedSecond.pop('ORIGIN_PT_LONGITUDE'))



# Creating DESTINATION_PT_LATITUDE & DESTINATION_PT_LONGITUDE column
dfCombinedSecond['DESTINATION_PT_LATITUDE'] = dfCombinedSecond['ORIGIN_PT_NAME'].map(latitude_mapping) 
dfCombinedSecond['DESTINATION_PT_LONGITUDE'] = dfCombinedSecond['ORIGIN_PT_NAME'].map(longitude_mappping) 


# Insert DESTINATION LATITUDE & LATITUDE next to their respective columns
locDestLatitude = dfCombinedSecond.columns.get_loc('DESTINATION_PT_CODE') + 1
locDestLongitude = dfCombinedSecond.columns.get_loc('DESTINATION_PT_CODE') + 2
dfCombinedSecond.insert(locDestLatitude, 'DESTINATION_PT_LATITUDE', dfCombinedSecond.pop('DESTINATION_PT_LATITUDE'))
dfCombinedSecond.insert(locDestLongitude, 'DESTINATION_PT_LONGITUDE', dfCombinedSecond.pop('DESTINATION_PT_LONGITUDE'))

### Mapping train lines to train codes

In [ ]:
train_line_mapping = {
    'EW': 0, # East-West Line
    'CG': 0, # East-West Line to Changi Airport
    'NS': 1, # North-South Line
    'NE': 2, # North-East Line
    'CC': 3, # Circle Line
    'CE': 3, # Circle Line (Bayfront, Marina Bay)
    'DT': 4, # Downtown Line
    'TE': 5, # Thomson-East Coast Line
    'BP': 6, # Bukit Panjang LRT
    'SW': 7, # Sengkang LRT West
    'SE': 7, # Sengkang LRT East
    'PW': 8, # Punggol LRT West
    'PE': 8, # Punggol LRT East
}

def map_train_codes(pt_code):
    # Initialize an empty list to store the mapped train codes
    train_codes = []
    # Split the pt_code by '/' and iterate over each part
    for code in pt_code.split('/'):
        # Iterate over each key in the mapping to find a match
        for key in train_line_mapping:
            # If the key is found at the start of the code segment, append the mapped value
            if code.startswith(key):
                train_codes.append(train_line_mapping[key])
                break  # Break the loop once the match is found
    return train_codes


# Apply the revised function to the ORIGIN_PT_CODE column
dfCombinedSecond['ORIGIN_TRAIN_CODES'] = dfCombinedSecond['ORIGIN_PT_CODE'].apply(map_train_codes)

# Convert the ORIGIN_PT_CODE column to a list
train_codes_list = dfCombinedSecond['ORIGIN_TRAIN_CODES'].tolist()

# Insert ORIGIN_TRAIN_CODES next to ORIGIN_TRAIN_LINES
loc = dfCombinedSecond.columns.get_loc('ORIGIN_TRAIN_LINES') + 1
dfCombinedSecond.insert(loc, 'ORIGIN_TRAIN_CODES', dfCombinedSecond.pop('ORIGIN_TRAIN_CODES'))



# Apply the revised function to the DESTINATION_PT_CODE column
dfCombinedSecond['DESTINATION_TRAIN_CODES'] = dfCombinedSecond['DESTINATION_PT_CODE'].apply(map_train_codes)

# Convert the DESTINATION_PT_CODE column to a list
train_codes_list = dfCombinedSecond['DESTINATION_TRAIN_CODES'].tolist()

# Insert DESTINATION_TRAIN_CODES next to DESTINATION_TRAIN_LINES
loc1 = dfCombinedSecond.columns.get_loc('DESTINATION_TRAIN_LINES') + 1
dfCombinedSecond.insert(loc1, 'DESTINATION_TRAIN_CODES', dfCombinedSecond.pop('DESTINATION_TRAIN_CODES'))


## Saving our Second Dataset

In [ ]:
# Saving our processed dataset as csv
output_path = './outputDatasets/trainStationData2.csv'
dfCombinedSecond.to_csv(output_path, index=False)

# The End of processing,

# Moving onto trainDataAnalysis!